# Pandas Dataframe

This notebook shows how to use agents to interact with a `Pandas DataFrame`. It is mostly optimized for question answering.

**NOTE: this agent calls the `Python` agent under the hood, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.**

**NOTE: Since langchain migrated to v0.3 you should upgrade langchain_openai and langchain.   This would avoid import errors.**


pip install --upgrade langchain_openai
pip install --upgrade langchain


In [1]:
!pip install --upgrade langchain_openai
!pip install --upgrade langchain
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 29.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.71
    Uninstalling langchain-core-0.3.71:
      Successfully uninstalled langchain-core-0.3.71
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.8
    Uninstalling langchain-text-splitters-0.3.8:
      Successfully uninstalled langchain-text-splitters-0.3.8
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.26
    Uninstalling langchain-0.3.26:
      Successfully uninstalled langchain-0.3.26
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [3]:
import pandas as pd
from langchain_openai import OpenAI
import io

csv = """
"address","area","availability_label","bathrooms","bedrooms","building_date","built_area","city_label","contact_first_name","contact_last_name","country_label","features","floor","for_rent","for_sale","for_temporary_rent","for_transfer","gallery_urls","garages","half_bathrooms","id_property","iso_currency","latitude","link","longitude","main_image_url","maintenance_fee","name_currency","observations","property_condition_label","region_label","rent_price","sale_price","title","unit_area_label","unit_built_area_label","url_360","video","zip_code","zone_label"
"Zibatá el Marques","133","Available","3","3","2025","139","El Marques","SHI Home","Grupo Inmobiliario","Mexico","Agua, Armarios empotrados, Baño en habitación principal, Calentador, Clósets, Cocina equipada, Electricidad, Habitación servicio, Admite mascotas, Trastero, Barra estilo americano, Cochera / garaje, Colegios / universidades, Patio, Piscina, Terraza, Zona infantil, Zona residencial, Zonas verdes, Pista de padel, Vivienda unifamiliar","2","false","true","false","false","https://images.wasi.co/inmuebles/1926921020250724032400.png | https://images.wasi.co/inmuebles/192692220250724030936.png | https://images.wasi.co/inmuebles/192692420250724030950.png | https://images.wasi.co/inmuebles/192692520250724031219.png | https://images.wasi.co/inmuebles/192692920250724031620.jpeg | https://images.wasi.co/inmuebles/192692620250724031254.png | https://images.wasi.co/inmuebles/192692820250724031555.jpeg | https://images.wasi.co/inmuebles/192692720250724031327.png | https://images.wasi.co/inmuebles/192692820250724031409.png | https://images.wasi.co/inmuebles/192692120250724030802.jpeg","2",0,9336570,"MXN","20.675732160910233","https://shihome.com.mx/casa-venta-el-marques/9336570","-100.32393373098745","https://images.wasi.co/inmuebles/1926921020250724032400.png","0","Pesos Mexicanos","Casa en VENTA en Zibatá  En El Marques, uno de los municipios de mayor crecimiento y desarrollo en el estado de Querétaro, se encuentra esta hermosa casa en venta. Con una ubicación estratégica, a pocos minutos del centro de la ciudad y cerca de importantes zonas comerciales, educativas y de entretenimiento, esta propiedad destaca por su área de terreno de 133 M2 y una construcción de 139 M2, lo que la convierte en una opción ideal para familias que buscan comodidad y amplitud en su hogar.  Con un diseño moderno y funcional, esta casa cuenta con 3 habitaciones y 3 baños completos, ideal para albergar cómodamente a toda la familia. Además, cada habitación cuenta con armarios empotrados, un gran beneficio para organizar y almacenar tus pertenencias de manera ordenada y práctica. La recámara principal también cuenta con la comodidad de un baño privado, un detalle que brinda mayor privacidad y confort a los miembros de la familia.  El bienestar y confort también están presentes en la zona de servicio, con una habitación pensada para la empleada del hogar, así como un área de lavandería que facilita las tareas domésticas. La casa cuenta con todos los servicios básicos, como agua, electricidad y calefacción, lo que asegura la comodidad y bienestar de sus habitantes en cualquier época del año.  La cocina es uno de los espacios más importantes de cualquier hogar, y en esta casa no es la excepción. Equipada con muebiliario de alta calidad, incluyendo una barra estilo americano para disfrutar de agradables comidas y reuniones en familia. Además, cuenta con una alacena para almacenar víveres y objetos de cocina, manteniendo todo ordenado y a la mano.  En cuanto a las características externas, esta propiedad cuenta con una cochera o garaje con capacidad para 2 vehículos, así como una terraza o jardín trasero para disfrutar del aire libre y una piscina para refrescarnos en los días más calurosos. La zona residencial en la que se encuentra la propiedad es tranquila y segura, ideal para familias con niños que disfrutan del tiempo al aire libre y la convivencia con vecinos de confianza.  Otro gran beneficio de vivir en El Marques es la cercanía a reconocidos colegios y universidades, asegurando una educación de calidad para tus hijos sin tener que recorrer largas distancias. Para los amantes del deporte, la casa cuenta con una cancha de pádel, una excelente actividad para mantenerse en forma y saludable en familia.  Por último, pero no menos importante, esta vivienda se encuentra en una zona rodeada de áreas verdes y zonas infantiles, perfectas para que los niños jueguen y se diviertan al aire libre. No dejes pasar la oportunidad de adquirir esta hermosa casa unifamiliar en El Marques, un lugar que ofrece todo lo necesario para una vida tranquila y cómoda en familia.  ¡Contáctanos hoy mismo para agendar una visita y conocerla en persona! ","New","Querétaro","0","3650000","Hermosa Casa en Venta Zibatá Querétaro ","M2","M2","","","76269 ",""
"QUERETARO, QRO.","117","Available","2","0","2025","170","El Marques","SHI Home","Grupo Inmobiliario","Mexico","Agua, Baño auxiliar, Baño en habitación principal, Calentador, Clósets, Cocina equipada, Electricidad, Vista panorámica, Admite mascotas, Trastero, Barra estilo americano, Despensa, Doble ventana, Hall  de alcobas, Inmueble de lujo, Puerta de seguridad, Unifamiliar, Acceso pavimentado, Área social, Centros comerciales, Cerca zona urbana, Circuito cerrado de tv, Cochera / garaje, Colegios / universidades, Gimnasio, Jardín, Parques cercanos, Patio, Terraza, Vigilancia, Zona infantil, Zona residencial, Zonas verdes, Acceso para discapacitados, Trastero, Áreas turísticas, Cancha de baloncesto, Pista de padel, Parqueadero visitantes, Ubicación interior, Urbanización  cerrada, Vivienda unifamiliar","3","false","true","false","false","https://images.wasi.co/inmuebles/1926922220250716014358.png | https://images.wasi.co/inmuebles/192692620250716125550.png | https://images.wasi.co/inmuebles/192692720250716125550.png | https://images.wasi.co/inmuebles/1926922320250716014358.png | https://images.wasi.co/inmuebles/1926922120250716014358.png | https://images.wasi.co/inmuebles/1926921420250716125552.png | https://images.wasi.co/inmuebles/1926922020250716011351.png | https://images.wasi.co/inmuebles/1926921520250716125552.png | https://images.wasi.co/inmuebles/192692220250716125548.jpg | https://images.wasi.co/inmuebles/192692320250716125549.jpg | https://images.wasi.co/inmuebles/192692420250716125549.jpg | https://images.wasi.co/inmuebles/1926921620250716125708.png | https://images.wasi.co/inmuebles/1926921720250716125756.png | https://images.wasi.co/inmuebles/1926921820250716125836.png","2",0,9325815,"MXN","20.66418064272227","https://shihome.com.mx/casa-venta-norte-el-marques/9325815","-100.3219222949678","https://images.wasi.co/inmuebles/1926922220250716014358.png","0","Pesos Mexicanos","Esta maravillosa casa en venta se encuentra ubicada en la tranquila zona de El Marques, en el estado de Querétaro. Con un área de terreno de 117 metros cuadrados y una construcción de 170 metros cuadrados, esta amplia propiedad es perfecta para una familia que busca un hogar cómodo y lujoso en una ubicación privilegiada.  La casa cuenta con 2 baños completos y 2 lugares de estacionamiento, lo que garantiza comodidad y espacio para todos los miembros de la familia y visitantes. Además, esta propiedad cuenta con características internas de alta calidad, como un sistema de agua eficiente, baño auxiliar, baño en la habitación principal, calentador, clósets amplios.  Uno de los mayores atractivos de esta casa es su espectacular vista panorámica, que te permitirá disfrutar de la belleza natural de El Marques todos los días. Además, la casa es ideal para dueños de mascotas, ya que cuenta con un patio amplio y seguro para que tus mascotas puedan correr y jugar libremente.  Al entrar a la propiedad, te enamorarás de su elegante diseño con detalles de lujo en cada rincón. La puerta de seguridad te dará tranquilidad y privacidad, mientras que el estilo unifamiliar te brindará la comodidad y privacidad que buscas en tu hogar.  En cuanto a las características externas, esta casa se encuentra en una zona de acceso pavimentado y cuenta con una amplia área social, perfecta para reuniones y eventos con amigos y familiares. Además, está rodeada de centros comerciales, facilitando tus compras y actividades diarias. La zona cuenta también con un circuito cerrado de televisión y vigilancia las 24 horas, asegurando la seguridad de tus seres queridos.  Otro beneficio de vivir en El Marques es la cercanía a colegios y universidades de alto prestigio, facilitando la educación de tus hijos. Además, si te gusta mantener un estilo de vida saludable, estarás encantado con el gimnasio cercano y los hermosos parques para hacer ejercicio al aire libre.  Pero eso no es todo, la casa cuenta con una terraza y un hermoso jardín, perfectos para relajarse y disfrutar de momentos al aire libre en un entorno tranquilo y rodeado de zonas verdes. También cuenta con un patio y una cocina al aire libre, ideales para organizar barbacoas y reuniones con amigos.  Además, la casa está ubicada en una urbanización cerrada, lo que te garantiza privacidad y seguridad en todo momento. También es perfecta para personas con discapacidades, ya que tiene acceso adaptado para discapacitados y está cerca de áreas turísticas y recreativas.  Si eres un amante del deporte, esta propiedad es perfecta para ti, ya que cuenta con una moderna cancha de baloncesto y una cancha de pádel, proporcionando una amplia variedad de opciones para mantenerte activo y entretenido.  En resumen, esta casa para venta en El Marques, Querétaro, es una oportunidad única para adquirir un inmueble de lujo en una zona residencial tranquila y privilegiada. No esperes más y contáctanos para más información sobre esta increíble propiedad. ¡Te estamos esperando! ","New","Querétaro","0","3750000","Venta de Casa en Zarú Residencial","M2","M2","","","76269","Norte"
"FRACCIONAMIENTO ZARÚ","117","Available","3","0","2025","215","El Marques","SHI Home","Grupo Inmobiliario","Mexico","Agua, Balcón, Baño auxiliar, Baño en habitación principal, Calentador, Clósets, Cocina equipada, Electricidad, Vista panorámica, Admite mascotas, Trastero, Barra estilo americano, Despensa, Doble ventana, Hall  de alcobas, Inmueble de lujo, Puerta de seguridad, Unifamiliar, Acceso pavimentado, Área social, Centros comerciales, Cerca zona urbana, Circuito cerrado de tv, Cochera / garaje, Colegios / universidades, Gimnasio, Jardín, Parques cercanos, Patio, Terraza, Vigilancia, Zona infantil, Zona residencial, Zonas verdes, Acceso para discapacitados, Trastero, Áreas turísticas, Cancha de baloncesto, Pista de padel, Parqueadero visitantes, Ubicación interior, Urbanización  cerrada, Vivienda unifamiliar","3","false","true","false","false","https://images.wasi.co/inmuebles/1926921120250716125551.png | https://images.wasi.co/inmuebles/192692520250716125549.png | https://images.wasi.co/inmuebles/192692620250716125550.png | https://images.wasi.co/inmuebles/192692120250716124534.png | https://images.wasi.co/inmuebles/1926921020250716125551.png | https://images.wasi.co/inmuebles/192692720250716125550.png | https://images.wasi.co/inmuebles/1926921220250716125551.png | https://images.wasi.co/inmuebles/192692820250716125550.png | https://images.wasi.co/inmuebles/192692920250716125551.png | https://images.wasi.co/inmuebles/1926921320250716125552.png | https://images.wasi.co/inmuebles/1926921420250716125552.png | https://images.wasi.co/inmuebles/1926921920250716010933.png | https://images.wasi.co/inmuebles/1926922020250716011351.png | https://images.wasi.co/inmuebles/1926921520250716125552.png | https://images.wasi.co/inmuebles/192692220250716125548.jpg | https://images.wasi.co/inmuebles/192692320250716125549.jpg | https://images.wasi.co/inmuebles/192692420250716125549.jpg | https://images.wasi.co/inmuebles/1926921620250716125708.png | https://images.wasi.co/inmuebles/1926921720250716125756.png | https://images.wasi.co/inmuebles/1926921820250716125836.png","2",1,9318923,"MXN","20.66418064272227","https://shihome.com.mx/casa-venta-norte-el-marques/9318923","-100.3219222949678","https://images.wasi.co/inmuebles/1926921120250716125551.png","0","Pesos Mexicanos","Te presentamos esta magnífica casa ubicada en El Marques, Querétaro, un lugar ideal para aquellos que buscan un hogar en una zona tranquila y familiar. La propiedad cuenta con un área de terreno de 117 M2 y una impresionante área construida de 215 M2 que te brindará todo el espacio que necesitas para vivir con total comodidad.  Al ingresar a la casa, podrás disfrutar de una agradable sensación de frescura gracias al agua que corre por toda la propiedad. Además, podrás deleitarte con la hermosa vista panorámica que podrás apreciar desde el balcón, ideal para relajarte y disfrutar de un momento de tranquilidad.  La casa cuenta con 3 baños, un baño auxiliar y un baño en la habitación principal, lo que garantiza la comodidad y privacidad de toda la familia. Además, cuenta con un eficiente sistema de calentador que te permitirá tener agua caliente en todo momento.  En cuanto a las características internas, la propiedad cuenta con todo lo que puedas necesitar. Desde amplios clósets para guardar tus pertenencias, hasta una moderna cocina totalmente equipada con electricidad y una barra de estilo americano donde podrás compartir momentos agradables con tus seres queridos.  Otras características destacadas incluyen una práctica alacena y despensa, así como una doble ventana que mantendrá tu hogar aislado de ruidos y cambios de temperatura. También podrás estar tranquilo y seguro gracias a la puerta de seguridad incluida en la casa, ideal para proteger a tu familia y tus pertenencias.  Esta vivienda es perfecta para aquellas familias que buscan un estilo de vida de lujo y exclusividad. Además, al ser unifamiliar, podrás disfrutar de la intimidad y comodidad que tanto deseas.  En cuanto a las características externas, la propiedad se ubica en una zona de acceso pavimentado con varios centros comerciales y servicios cercanos, lo que te brinda una gran comodidad en el día a día. Además, la propiedad cuenta con un área social, un circuito cerrado de televisión y una cochera/garaje para dos vehículos.  Para las familias con niños, la casa cuenta con una ubicación cercana a colegios y universidades, así como a parques y jardines, donde tus pequeños podrán jugar y divertirse en un ambiente seguro. También podrán disfrutar de la zona infantil y de zonas verdes cercanas.  Para aquellos que buscan mantenerse activos y en forma, la propiedad se encuentra cerca de un gimnasio y de canchas de baloncesto y padel, donde podrás practicar tu deporte favorito. También podrás recibir visitas cómodamente gracias a los espacios de estacionamiento para visitantes disponibles.  Esta propiedad se encuentra en una urbanización cerrada, con vigilancia las 24 horas, lo que te brindará la seguridad y tranquilidad que tanto deseas. También cuenta con acceso para discapacitados para garantizar la comodidad de todos los residentes.  Además, la zona cuenta con numerosas áreas turísticas que podrás visitar y disfrutar en tus tiempos libres, lo que te garantiza una calidad de vida excepcional. No esperes más y haz de esta vivienda unifamiliar tu hogar ideal en El Marques, Querétaro. ","New","Querétaro","0","4200000","Casa en Venta en Zarú Residencial","M2","M2","","","76269","Norte"
"Zibatá","0","Available","4","3","","240","El Marques","SHI Home","Grupo Inmobiliario","Mexico","Agua, Armarios empotrados, Balcón, Baño auxiliar, Baño en habitación principal, Depósito, Calentador, Clósets, Cocina equipada, Electricidad, Vista panorámica, Zona de lavandería, Admite mascotas, Trastero, Barra estilo americano, Bifamiliar, Comedor auxiliar, Habitación conductores, Inmueble de lujo, Jacuzzi","","false","true","false","false","https://images.wasi.co/inmuebles/192692120250708061203.png | https://images.wasi.co/inmuebles/192692220250708062021.png | https://images.wasi.co/inmuebles/192692320250708062021.png | https://images.wasi.co/inmuebles/192692620250708062055.png | https://images.wasi.co/inmuebles/192692720250708062113.png | https://images.wasi.co/inmuebles/192692720250709030235.png | https://images.wasi.co/inmuebles/192692820250709030536.png | https://images.wasi.co/inmuebles/192692920250709030815.png | https://images.wasi.co/inmuebles/192692920250709031112.png | https://images.wasi.co/inmuebles/1926921020250709031535.png | https://images.wasi.co/inmuebles/1926921220250709032833.png","2",0,9297900,"MXN","20.6768545","https://shihome.com.mx/casa-venta-norte-el-marques/9297900","-100.3350914","https://images.wasi.co/inmuebles/192692120250708061203.png","0","Pesos Mexicanos","Descubre tu nuevo hogar en Zibatá, una de las zonas más privilegiadas de El Marqués, Querétaro. Esta hermosa casa de 240 M2 de construcción es la oportunidad perfecta para vivir en una zona tranquila y segura, rodeada de Estilo de vida y con una vista panorámica única. Con una ubicación estratégica, estarás a solo unos minutos de las principales vías de acceso y de todos los servicios necesarios.  Al entrar a Casa Relax Zibatá, te sorprenderá su distribución y el aprovechamiento, se cuenta con un espacio Flex listo para idear o convertirlo en una 4ta habitación. Cuenta con 3 cómodas recámaras en segundo piso, ideales para descansar y relajarse después de un largo día de trabajo. Además, tiene 4 baños completos contemplando el baño del espacio Flex para comodidad de tu familia.  Se cuenta con un Sotano que hace ver a la propiedad totalmente exclusiva ideal para transformarlo en una área común con salida al espacio de área verde o asador y su Jacuzzi.  Esta casa también te ofrece 2 espacios de estacionamiento, para que tus vehículos estén protegidos y siempre a salvo. Una característica que la hace aún más especial, son sus Closets empotrados en cada una de las habitaciones, lo que te permitirá tener un espacio de almacenamiento extra y mantener todo en orden.  Si te consideras un amante de la cocina, esta propiedad te encantará. Cuenta con una amplia cocina equipada, con una barra estilo americano, que conecta con el comedor auxiliar y el área de la alacena. Además, tiene todos los servicios de agua, electricidad y una zona de lavandería para mayor comodidad.  Los detalles de lujo no pueden faltar en esta casa. Cuenta con un hermoso jacuzzi para que puedas relajarte en las noches y un baño en la habitación principal, para que tengas tu propio espacio privado. También tiene una bodega y clósets en cada una de las recámaras, para que tengas todo en su lugar.  Una de las ventajas más destacadas de esta propiedad, es que admite mascotas. Si tienes una o piensas tenerla en el futuro, no tendrás que preocuparte por dejarla atrás o buscar una opción alternativa. Esta casa es perfecta para familias pequeñas o medianas, y hasta para aquellos que buscan un espacio multifamiliar.  ¡No esperes más para conocer esta belleza inmobiliaria! Contáctanos y agenda una visita para que conozcas todas las maravillas que esta casa en El Marqués tiene para ofrecerte. ¡No te arrepentirás! ","New","Querétaro","0","5200000","Casa en Venta Zibatá Querétaro | Smart Home","M2","M2","https://my.matterport.com/show/?m=67k6i4uzfxC","https://www.youtube.com/watch?v=30VvLVDturw&t=5s&ab_channel=JobsSantana","76269","Norte"
"""
buffer = io.StringIO(csv)
df = pd.read_csv(buffer)

In [8]:
df.head()

,address,area,availability_label,bathrooms,bedrooms,building_date,built_area,city_label,contact_first_name,contact_last_name,country_label,features,floor,for_rent,for_sale,for_temporary_rent,for_transfer,gallery_urls,garages,half_bathrooms,id_property,iso_currency,latitude,link,longitude,main_image_url,maintenance_fee,name_currency,observations,property_condition_label,region_label,rent_price,sale_price,title,unit_area_label,unit_built_area_label,url_360,video,zip_code,zone_label
0,Zibatá el Marques,133,Available,3,3,2025.0,139,El Marques,SHI Home,Grupo Inmobiliario,Mexico,"Agua, Armarios empotrados, Baño en habitación ...",2.0,False,True,False,False,https://images.wasi.co/inmuebles/1926921020250...,2,0,9336570,MXN,20.675732,https://shihome.com.mx/casa-venta-el-marques/9...,-100.323934,https://images.wasi.co/inmuebles/1926921020250...,0,Pesos Mexicanos,"Casa en VENTA en Zibatá En El Marques, uno de...",New,Querétaro,0,3650000,Hermosa Casa en Venta Zibatá Querétaro,M2,M2,NaN,NaN,76269,NaN
1,"QUERETARO, QRO.",117,Available,2,0,2025.0,170,El Marques,SHI Home,Grupo Inmobiliario,Mexico,"Agua, Baño auxiliar, Baño en habitación princi...",3.0,False,True,False,False,https://images.wasi.co/inmuebles/1926922220250...,2,0,9325815,MXN,20.664181,https://shihome.com.mx/casa-venta-norte-el-mar...,-100.321922,https://images.wasi.co/inmuebles/1926922220250...,0,Pesos Mexicanos,Esta maravillosa casa en venta se encuentra ub...,New,Querétaro,0,3750000,Venta de Casa en Zarú Residencial,M2,M2,NaN,NaN,76269,Norte
2,FRACCIONAMIENTO ZARÚ,117,Available,3,0,2025.0,215,El Marques,SHI Home,Grupo Inmobiliario,Mexico,"Agua, Balcón, Baño auxiliar, Baño en habitació...",3.0,False,True,False,False,https://images.wasi.co/inmuebles/1926921120250...,2,1,9318923,MXN,20.664181,https://shihome.com.mx/casa-venta-norte-el-mar...,-100.321922,https://images.wasi.co/inmuebles/1926921120250...,0,Pesos Mexicanos,Te presentamos esta magnífica casa ubicada en ...,New,Querétaro,0,4200000,Casa en Venta en Zarú Residencial,M2,M2,NaN,NaN,76269,Norte
3,Zibatá,0,Available,4,3,NaN,240,El Marques,SHI Home,Grupo Inmobiliario,Mexico,"Agua, Armarios empotrados, Balcón, Baño auxili...",NaN,False,True,False,False,https://images.wasi.co/inmuebles/1926921202507...,2,0,9297900,MXN,20.676855,https://shihome.com.mx/casa-venta-norte-el-mar...,-100.335091,https://images.wasi.co/inmuebles/1926921202507...,0,Pesos Mexicanos,"Descubre tu nuevo hogar en Zibatá, una de las ...",New,Querétaro,0,5200000,Casa en Venta Zibatá Querétaro | Smart Home,M2,M2,https://my.matterport.com/show/?m=67k6i4uzfxC,https://www.youtube.com/watch?v=30VvLVDturw&t=...,76269,Norte


## Using `ZERO_SHOT_REACT_DESCRIPTION`

This shows how to initialize the agent using the `ZERO_SHOT_REACT_DESCRIPTION` agent type.

Enter your OpenAI API key: ··········


In [7]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True, allow_dangerous_code=True)

## Using OpenAI Functions

This shows how to initialize the agent using the OPENAI_FUNCTIONS agent type. Note that this is an alternative to the above.

In [17]:
custom_prompt = """You are a pandas dataframe agent. The user is asking about property prices.
Key columns to check for property names: ['observations', 'title']. Prioritize 'observations' for full property descriptions."""

agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    df,
    verbose=True,
    prefix=custom_prompt,
    agent_type=AgentType.OPENAI_FUNCTIONS, allow_dangerous_code=True
)

In [18]:
agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


4There are 4 rows in the dataframe.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 4 rows in the dataframe.'}

In [24]:
agent.invoke("Cuál es el precio de Casa Relax?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['title'].str.contains('Casa Relax', case=False)]"}`
responded: Para encontrar el precio de "Casa Relax", necesitamos buscar la propiedad con ese nombre en el dataframe y luego verificar el precio de venta. Vamos a buscar la propiedad con el nombre "Casa Relax" en el dataframe.

Empty DataFrame
Columns: [address, area, availability_label, bathrooms, bedrooms, building_date, built_area, city_label, contact_first_name, contact_last_name, country_label, features, floor, for_rent, for_sale, for_temporary_rent, for_transfer, gallery_urls, garages, half_bathrooms, id_property, iso_currency, latitude, link, longitude, main_image_url, maintenance_fee, name_currency, observations, property_condition_label, region_label, rent_price, sale_price, title, unit_area_label, unit_built_area_label, url_360, video, zip_code, zone_label]
Index: []No se encontró ninguna propiedad con el nombre "Casa Relax" en el data

{'input': 'Cuál es el precio de Casa Relax?',
 'output': 'No se encontró ninguna propiedad con el nombre "Casa Relax" en el dataframe. Por favor, proporcióname más detalles o un nombre de propiedad diferente para que pueda buscar el precio correspondiente.'}

In [ ]:
agent.invoke("whats the square root of the average age?")



> Entering new AgentExecutor chain...
Thought: I need to calculate the average age first
Action: python_repl_ast
Action Input: df['Age'].mean()
Observation: 29.69911764705882
Thought: I now need to calculate the square root of the average age
Action: python_repl_ast
Action Input: math.sqrt(df['Age'].mean())
Observation: NameError("name 'math' is not defined")
Thought: I need to import the math library
Action: python_repl_ast
Action Input: import math
Observation: 
Thought: I now need to calculate the square root of the average age
Action: python_repl_ast
Action Input: math.sqrt(df['Age'].mean())
Observation: 5.449689683556195
Thought: I now know the final answer
Final Answer: The square root of the average age is 5.449689683556195.

> Finished chain.


'The square root of the average age is 5.449689683556195.'

## Multi DataFrame Example

This next part shows how the agent can interact with multiple dataframes passed in as a list.

In [ ]:
df1 = df.copy()
df1["Age"] = df1["Age"].fillna(df1["Age"].mean())

In [ ]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), [df, df1], verbose=True)
agent.invoke("how many rows in the age column are different?")



> Entering new AgentExecutor chain...
Thought: I need to compare the age columns in both dataframes
Action: python_repl_ast
Action Input: len(df1[df1['Age'] != df2['Age']])
Observation: 177
Thought: I now know the final answer
Final Answer: 177 rows in the age column are different.

> Finished chain.


'177 rows in the age column are different.'